# Notebook for statistical  and time series analysis of the dataset "NYC Yellow Taxi Trip Data" 

### Install the required packages, lunch this cell only once (code 1)


In [ ]:
%pip install kagglehub
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install seaborn
%pip install folium 
%pip install scipy
%pip install -U jupyter ipywidgets
%pip install -U jupyterlab-widgets
import seaborn as sns
from scipy.stats import norm, alpha
import numpy as np
import kagglehub
import pandas as pd
import os
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, MarkerCluster
from datetime import datetime
from scipy.stats import probplot

## Download the dataset and load the data
##### lunch this cell only once (code 2)

In [ ]:
# Download latest version
path = kagglehub.dataset_download("elemento/nyc-yellow-taxi-trip-data")
# print("Path to dataset files:", path) use only for debug
file_path1 = os.path.join(path, "yellow_tripdata_2015-01.csv")  # Path to the dataset file
file_path2 = os.path.join(path, "yellow_tripdata_2016-01.csv")  # Path to the dataset file
file_path3 = os.path.join(path, "yellow_tripdata_2016-02.csv")  # Path to the dataset file
file_path4 = os.path.join(path, "yellow_tripdata_2016-03.csv")  # Path to the dataset file

#### there is un error in un colum, so you must run this script for  update and change the column name in first dataset (if you need this column)  (code 3)

In [ ]:
def update_and_change(file_path):
    df = pd.read_csv(file_path)
    # Update the column name
    if 'RateCodeID' in df.columns:
        df.rename(columns={'RateCodeID': 'RatecodeID'}, inplace=True)
    return df

df_2015_01 = update_and_change(file_path1)

#####  Check if database was imported correctly, this script is optional will be used only for debug (code 4)

In [ ]:
# Check if the file exists
if os.path.exists(file_path1):
    # upload the file to the notebook
    df = pd.read_csv(file_path1)

    # Print the colum names
    print("Nomi delle colonne nel dataset:", df.columns.tolist())
else:
    print(f"Il file non esiste. Controlla il percorso: {file_path1}")


### Statistical analysis in colum VendorID for the dataset

##### Union of the dataset and creation of the bar plot for the frequency of the VendorID (code 5)

In [ ]:
#! before run this you must run the code 1 and 2
vendor_colum = ['VendorID']
df_2015_01 = pd.read_csv(file_path1, usecols=vendor_colum)    # Load the dataset  in a DataFrame
df_2016_01 = pd.read_csv(file_path2, usecols=vendor_colum)    # Load the dataset  in a DataFrame
df_2016_02 = pd.read_csv(file_path3, usecols=vendor_colum)    # Load the dataset  in a DataFrame
df_2016_03 = pd.read_csv(file_path4, usecols=vendor_colum)    # Load the dataset  in a DataFrame
print("Number of rows in the dataset 2015-01:", len(df_2015_01))
print("Number of rows in the dataset 2016-01:", len(df_2016_01))
print("Number of rows in the dataset 2016-02:", len(df_2016_02))
print("Number of rows in the dataset 2016-03:", len(df_2016_03))

# Concatenate the dataframes
df = pd.concat([df_2015_01, df_2016_01, df_2016_02, df_2016_03], ignore_index=True)

#### Creation of the bar plot  and pie cacke for the frequency of the VendorID (code 6)

In [ ]:

# set a default style 
plt.style.use('default')

# calculate frequenzy of 
fr_vendor = df['VendorID'].value_counts()

# Creazione del grafico a barre con annotazioni e sfondo bianco
fig, ax = plt.subplots()
fig.patch.set_facecolor('white')  # Impostiamo il colore di sfondo della figura
ax.set_facecolor('white')  # Impostiamo il colore di sfondo dell'area del grafico

# Creazione del grafico a barre
fr_vendor.plot(kind='bar', color=['#1f77b4', '#2ca02c'], ax=ax)

# Aggiunta di annotazioni numeriche per ogni barra
for i, count in enumerate(fr_vendor):
    ax.text(i, count + 5, str(count), ha='center', va='bottom', fontsize=10, fontweight='bold')

# Aggiunta di titolo e etichette
ax.set_title("Frequency of VendorID", fontsize=14, fontweight='bold')
ax.set_xlabel("VendorID", fontsize=12)
ax.set_ylabel("Number of trips", fontsize=12)
ax.set_xticks(range(len(fr_vendor.index)))
ax.set_xticklabels(fr_vendor.index, rotation=0)

# Visualizzazione del grafico a barre
plt.show()

# Creazione del grafico a torta con percentuali e conteggi, sfondo bianco
plt.figure(figsize=(8, 8), facecolor='white')  # Impostiamo il colore di sfondo della figura
fr_vendor.plot(kind='pie', 
               autopct=lambda p: f'{p:.1f}% ({int(p * fr_vendor.sum() / 100)})',  # Percentuali con conteggi
               colors=['#1f77b4', '#2ca02c'], 
               startangle=90, 
               counterclock=False, 
               wedgeprops=dict(width=0.3))  # Differenziazione dello stile con spessore

# Aggiunta di titolo
plt.title("Percentage of trips by VendorID", fontsize=14, fontweight='bold')

# Visualizzazione del grafico a torta
plt.show()


### Analysis of the distribution of the trips by hour (code 7)

In [ ]:
#! before run this you must run the code 1 and 2 and 3
# union in a single csv only with the column RateCodeID
colum_rate = ['RatecodeID'] 
df_2015_01_filtered = df_2015_01[['RatecodeID']]    # Load the dataset  in a DataFrame
df_2016_01 = pd.read_csv(file_path2, usecols=colum_rate)    # Load the dataset  in a DataFrame
df_2016_02 = pd.read_csv(file_path3, usecols=colum_rate)    # Load the dataset  in a DataFrame
df_2016_03 = pd.read_csv(file_path4, usecols=colum_rate)    # Load the dataset  in a DataFrame
df=pd.concat([df_2015_01_filtered, df_2016_01, df_2016_02, df_2016_03], ignore_index=True)



Creation of the bar plot for the frequency of the RateCodeID (code 8)

In [ ]:
# Creation of the bar plot for the frequency of the RateCodeID
df['Validity_RateCodeID'] = df['RatecodeID'].apply(lambda x: 'valid' if x in range(1, 7) else 'not valid')

# Filter the valid RateCodeID
ratecode_validi = df[df['Validity_RateCodeID'] == 'valid']['RatecodeID']

# Calculate the frequency of the RateCodeID
frequent_rate_id = ratecode_validi.value_counts()
frequent_rate_id['not valid'] = df['Validity_RateCodeID'].value_counts().get('not valid', 0)

# Plot the bar plot
plt.figure(figsize=(10, 6))
ax = frequent_rate_id.plot(kind='bar', color=['skyblue' if idx != 'not valid' else 'salmon' for idx in frequent_rate_id.index])

# Add title and labels
plt.title("Frequenza dei Codici Tariffari (RatecodeID)")
plt.xlabel("RatecodeID")
plt.ylabel("Numero di corse")
plt.xticks(rotation=0)

# Add annotations for each bar
for i, count in enumerate(frequent_rate_id):
    ax.text(i, count + max(frequent_rate_id) * 0.01, str(count), ha='center', va='bottom', fontsize=10)

# Show the bar plot
plt.show()

# add a pie chart for the % of the RateCodeID
plt.figure(figsize=(8, 8), facecolor='white')  
frequent_rate_id.plot(kind='pie', 
                      autopct=lambda p: f'{p:.1f}% ({int(p * frequent_rate_id.sum() / 100)})',
                      colors=['skyblue' if idx != 'Non valido' else 'salmon' for idx in frequent_rate_id.index], 
                      startangle=90, 
                      counterclock=False, 
                      wedgeprops=dict(width=0.3))  

plt.title("Percentuale delle corse per RatecodeID", fontsize=14, fontweight='bold')
plt.show()

print(frequent_rate_id)


In [ ]:
colum = ['tpep_pickup_datetime'] 
df_2015_01_filtered = pd.read_csv(file_path1, usecols=colum)    # Load the dataset  in a DataFrame
df_2016_01 = pd.read_csv(file_path2, usecols=colum)     
df_2016_02 = pd.read_csv(file_path3, usecols=colum)
df_2016_03 = pd.read_csv(file_path4, usecols=colum)

df_total = pd.concat([df_2015_01_filtered, df_2016_01, df_2016_02, df_2016_03], ignore_index=True)

# convert the column to datetime
df_total['tpep_pickup_datetime'] = pd.to_datetime(df_total['tpep_pickup_datetime'])

# extract the hour from the datetime
df_total['hour'] = df_total['tpep_pickup_datetime'].dt.hour

#couht the frequency of each hour
hour_freq = df_total['hour'].value_counts().sort_index()

# create array with the hours
hours=hour_freq.index
counts=hour_freq.values


#print frequency of each hour
print(hour_freq)

#plot for hour distribution of trips
plt.figure(figsize=(10, 6))
sns.barplot(x=hours, y=counts, color='skyblue')
plt.title("Distriubution of trips by hour")
plt.xlabel("hour")
plt.ylabel("Number of trips")
plt.xticks(np.arange(0, 24, 1))
plt.show()


### Analysis of the distribution of the trips by hour in a day

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

colum = ['tpep_pickup_datetime']  # scelgo la colonna tpep_pickup_datetime
df_2015_01_filtered = pd.read_csv(file_path1, usecols=colum)
df_2016_01 = pd.read_csv(file_path2, usecols=colum)
df_2016_02 = pd.read_csv(file_path3, usecols=colum)
df_2016_03 = pd.read_csv(file_path4, usecols=colum)

df_total = pd.concat([df_2015_01_filtered, df_2016_01, df_2016_02, df_2016_03], ignore_index=True)

df_total['tpep_pickup_datetime'] = pd.to_datetime(df_total['tpep_pickup_datetime'])

df_total['day'] = df_total['tpep_pickup_datetime'].dt.date
df_total['hour'] = df_total['tpep_pickup_datetime'].dt.hour

hourly_day_freq = df_total.groupby(['day', 'hour']).size().reset_index(name='count')

avg_hourly_by_day = hourly_day_freq.groupby('hour')['count'].mean().reset_index(name='avg_count')

print(avg_hourly_by_day)

plt.figure(figsize=(10, 6))
sns.barplot(x=avg_hourly_by_day['hour'], y=avg_hourly_by_day['avg_count'], color='skyblue')
plt.title("Distribution of trips by hour in a day")
plt.xlabel("Hour")
plt.ylabel("Average number of trips")
plt.xticks(np.arange(0, 24, 1))
plt.show()
unique_days = df_total['day'].unique()

unique_days_sorted = np.sort(unique_days)

print("Giorni trovati nel dataset:")
for day in unique_days_sorted:
    print(day)

### Try to fit the distribution of the trips by hour with a Gaussian distribution (code 9)


In [ ]:
hour_freq = df_total['hour'].value_counts().sort_index()

# 1. Standardizzazione dei dati
mean = hour_freq.mean()  # Calculate the mean
std = hour_freq.std()    # Deviazione standard for frequency of trips
# This process can traform the data for have a media of 0 and a standard deviation of 1


hour_freq_standardized = (hour_freq - mean) / std # Standardizzazione of the frequency of trips

# 2. Q-Q Plot for check the normality of the data
# Q-Q Plot (Quantile-Quantile Plot) compares the quantiles of your observed data (in this case, hour_freq_standardized) with the quantiles of a theoretical normal distribution.
# If your data follows a normal distribution:
# The Q-Q Plot points line up along the red diagonal line (representing the theoretical values of a standardized normal).
# If the data deviate from the diagonal line:
# Indicates that the data do not follow a normal distribution.
# The further the points are from the diagonal line, the more the data deviate from a normal distribution.
plt.figure(figsize=(10, 6))
probplot(hour_freq_standardized, dist="norm", plot=plt)
plt.title("Q-Q To check the normality of the data")
plt.grid(True)
plt.show()

#### Calculate the mean and the standard deviation of the distribution of the trips by hour (code 9)

In [ ]:
mu, std = norm.fit(df_total['hour'])

# create the gaussian distribution
xmin,xmax=0,23
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)

plt.figure(figsize=(10, 6))
plt.plot(x,p,'r-', lw=2 ,label='Gaussian fit' ) # Gaussian fit
sns.histplot(df_total['hour'], bins=24, color='skyblue', edgecolor='black', stat='density', kde=False,alpha=0.6)
#order the x axis and put the center of gayssian distribution in the middle
plt.xlim(xmin,xmax)
plt.title("Distribution of trips by hour")
plt.xlabel("Hour")
plt.ylabel("Density")
plt.xticks(np.arange(0, 24, 1))
plt.show()
# plt.legend()

#### barplot follow median and standard deviation (code 10)

In [ ]:
colum = ['tpep_pickup_datetime'] # scelgo la colonna tpep_pickup_datetime
df_2015_01_filtered = pd.read_csv(file_path1, usecols=colum)    # Load the dataset  in a DataFrame
df_2016_01 = pd.read_csv(file_path2, usecols=colum)     
df_2016_02 = pd.read_csv(file_path3, usecols=colum)
df_2016_03 = pd.read_csv(file_path4, usecols=colum)

df_total = pd.concat([df_2015_01_filtered, df_2016_01, df_2016_02, df_2016_03], ignore_index=True)

# convert the column to datetime
df_total['tpep_pickup_datetime'] = pd.to_datetime(df_total['tpep_pickup_datetime'])

# extract the hour from the datetime
df_total['hour'] = df_total['tpep_pickup_datetime'].dt.hour
#calculate trip for hour
hour_freq = df_total['hour'].value_counts().sort_index()
#plot for hour distribution of trips
plt.figure(figsize=(10, 6))
sns.barplot(x=hour_freq.index, y=hour_freq.values, color='skyblue')
plt.title("Distriubution of trips by hour")
plt.xlabel("hour")
plt.ylabel("Number of trips")
plt.xticks(np.arange(0, 24, 1))
plt.show()

### anlaisys on  drip distance (code 11)

In [ ]:
#! before run this you must run the code 1 and 2

plt.style.use('default')  
sns.set_theme(style="white")  # Set the style of the plots

# Load the dataset in a DataFrame
columns=['trip_distance']
df_2015_01 = pd.read_csv(file_path1, usecols=columns)   # Load the dataset  in a DataFrame  
df_2016_01 = pd.read_csv(file_path2, usecols=columns)
df_2016_02 = pd.read_csv(file_path3, usecols=columns)
df_2016_03 = pd.read_csv(file_path4, usecols=columns)

# Filter date with trip distance between 0 and 100 miles
df_total = pd.concat([df_2015_01, df_2016_01, df_2016_02, df_2016_03], ignore_index=True)
df_total_filtered = df_total[(df_total['trip_distance'] > 0) & (df_total['trip_distance'] <= 30)]

# Creare un intervallo personalizzato per i bin
custom_bins = np.arange(0, 21, 1)  # Bin da 0 a 20 con intervallo di 1 miglio

# Plot the histogram with custom bins
plt.figure(figsize=(16, 8))
counts, bins, bars = plt.hist(df_total_filtered['trip_distance'], bins=custom_bins, color='skyblue', edgecolor='black', alpha=0.6)

# Annotare ogni barra con il numero di viaggi in orientamento verticale
for count, bar in zip(counts, bars):
    height = bar.get_height()
    if height > 0:
        plt.text(
            bar.get_x() + bar.get_width() / 2,  # Posizione al centro della barra
            height,                             # Posizione in altezza sopra la barra
            f'{int(count)}',                    # Testo da visualizzare
            ha='center',                        # Allineamento orizzontale
            va='bottom',                        # Allineamento verticale
            fontsize=10,
            rotation=90                         # Rotazione verticale
        )

# Personalizzare il grafico
plt.title("Distribuzione delle distanze dei viaggi (fino a 20 miglia) con intervalli di 1 miglio")
plt.xlabel("Distanza del viaggio (miglia)")
plt.ylabel("Numero di viaggi")
plt.xticks(custom_bins)  # Mostrare tutti i tick corrispondenti ai bin
plt.show()


In [ ]:
# Classify trip distances into categories
bins = [0,5, 10,15, 20, 30]
labels = ['0-5 miglia', '5-10 miglia', '10-15 miglia', '15-20 miglia', '20-30 miglia']
df_total_filtered.loc[:, 'Range'] = pd.cut(df_total_filtered['trip_distance'], bins=bins, labels=labels, right=False)

# Calculate the count for each range
trip_counts = df_total_filtered['Range'].value_counts()

# Create an explode array to emphasize small slices
explode = [0.01 if value / trip_counts.sum() < 0.05 else 0 for value in trip_counts]

# Plot the pie chart
plt.figure(figsize=(10, 8))
wedges, texts, autotexts = plt.pie(
    trip_counts, 
    labels=trip_counts.index, 
    autopct='%.2f%%', 
    startangle=140, 
    colors=sns.color_palette("pastel"),
    explode=explode,  # Emphasize small slices
    pctdistance=0.85  # Adjust the position of percentage labels
)

# Improve text readability
for text in texts + autotexts:
    text.set_fontsize(10)

# Add lines connecting labels to small slices
plt.title("Distribuzione percentuale delle distanze dei viaggi", fontsize=14)
plt.tight_layout()
plt.show()

### Analysis of the distribution of the trip duration by the hours (code 12)

In [ ]:
#! before run this you must run the code 1, 2  and 3
# #unite all dataset in a single csv
file_paths=[file_path2, file_path3, file_path4]
def load_columns(file_paths, columns_envolved):
    df_list = []
    for file_path in file_paths:
        # load only the columns of interest
        df = pd.read_csv(file_path, usecols=columns_envolved)
        df_list.append(df)
    # unite all the dataframes
    return pd.concat(df_list, ignore_index=True)
df_distance = load_columns(file_paths, columns_envolved=['trip_distance'])
df_time_fase = load_columns(file_paths, columns_envolved=['trip_distance', 'tpep_pickup_datetime'])

In [ ]:
# 3. Cluster the data by the hour

df_ratecode_parzial = load_columns(file_paths, columns_envolved=['trip_distance', 'RatecodeID', 'tpep_pickup_datetime'])
df_2015_01_filtered = df_2015_01[['trip_distance', 'RatecodeID', 'tpep_pickup_datetime']]    # Load the dataset  in a DataFrame
df_ratecode=pd.concat([df_2015_01_filtered, df_ratecode_parzial], ignore_index=True)
# convert the column to datetime
df_ratecode['tpep_pickup_datetime'] = pd.to_datetime(df_ratecode_parzial['tpep_pickup_datetime'])
df_ratecode['pickup_hour'] = df_ratecode['tpep_pickup_datetime'].dt.hour

# Calculate the number of trips for each hour
trip_for_hour = df_ratecode.groupby('pickup_hour').size()

### code 13

In [ ]:
# Calcolare la distanza media per ora
distance_media_for_hour = df_ratecode.groupby('pickup_hour')['trip_distance'].mean()
# Creare la figura e il layout per i grafici
plt.figure(figsize=(12, 6))

# Grafico della distanza media per ora
plt.subplot(2, 1, 1)
distance_media_for_hour.plot(kind='line', color='orange', marker='o', linestyle='-', linewidth=2, markersize=6)
plt.title('Distanza Media per Ora', fontsize=14)
plt.xlabel('Ora del Giorno', fontsize=12)
plt.ylabel('Distanza Media del Viaggio (miglia)', fontsize=12)
plt.xticks(rotation=0)
plt.grid(True)

# Impostazioni layout
plt.tight_layout()

# Mostrare il grafico
plt.show()

### Crete realtions with ratecode and trip distance (code 14)

### map of the trips (code 15)
# exercise with folium

In [ ]:
columns_needed = [
    'tpep_pickup_datetime', 'pickup_longitude', 'pickup_latitude',
    'dropoff_longitude', 'dropoff_latitude'
]

df_2015_01 = pd.read_csv(file_path1, usecols=columns_needed)   # Load the dataset  in a DataFrame
df_2016_01 = pd.read_csv(file_path2, usecols=columns_needed)
df_2016_02 = pd.read_csv(file_path3, usecols=columns_needed)
df_2016_03 = pd.read_csv(file_path4, usecols=columns_needed)
#concate all the dataframes
df= pd.concat([df_2015_01, df_2016_01, df_2016_02, df_2016_03], ignore_index=True)

#convert the column to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# define the period of the day
def get_hour_period(hour):
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    elif 18 <= hour < 24:
        return 'evening'
    else:
        return 'night'

# create a column for the pickup period
df['pickup_period'] = df['tpep_pickup_datetime'].dt.hour.apply(get_hour_period)

# Filter the data only for the morning period
df = df[(df['pickup_latitude'] != 0) & (df['pickup_longitude'] != 0) &
        (df['dropoff_latitude'] != 0) & (df['dropoff_longitude'] != 0)]


In [ ]:
# Create the map centered on New York City
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Add a cluster to group the points
marker_cluster = MarkerCluster().add_to(nyc_map)

# Group the most frequent routes
for period in df['pickup_period'].unique():
    period_data = df[df['pickup_period'] == period]
    #   Group the most frequent routes
    frequent_routes = period_data.groupby(['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']).size().reset_index(name='counts')
    top_routes = frequent_routes.nlargest(40, 'counts') 
    
    # Add the routes, the start and end markers on the map
    for _, row in top_routes.iterrows():
        folium.PolyLine(
            locations=[(row['pickup_latitude'], row['pickup_longitude']), (row['dropoff_latitude'], row['dropoff_longitude'])],
            color='blue' if period == 'morning' else 'green' if period == 'afternoon' else 'orange' if period == 'evening' else 'purple',
            weight=20,
            opacity=1  
        ).add_to(marker_cluster)


# Show the map
nyc_map

### Antoher tipe of maps (code 16) Clusterizzazione per Tratte

In [ ]:
# Filter only the data for the morning
morning_data = df[df['pickup_period'] == 'morning']

# create a map centered on New York City
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# add a cluster to group the points
marker_cluster = MarkerCluster().add_to(nyc_map)

# gruop the most frequent routes
frequent_routes = morning_data.groupby(
    ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']
).size().reset_index(name='counts')
top_routes = frequent_routes.nlargest(40, 'counts')  # Prendiamo le 40 tratte più comuni

# add the routes, the start and end markers on the map
for _, row in top_routes.iterrows():
    # trip
    folium.PolyLine(
        locations=[
            (row['pickup_latitude'], row['pickup_longitude']),
            (row['dropoff_latitude'], row['dropoff_longitude'])
        ],
        color='blue',  # Colore per il tragitto
        weight=5,      # Spessore della linea
        opacity=0.7    # Opacità per il tragitto
    ).add_to(marker_cluster)

    # Marker pickup
    folium.Marker(
        location=(row['pickup_latitude'], row['pickup_longitude']),
        icon=folium.Icon(color='blue', icon='play', prefix='fa'),  # Icona per identificare il punto di partenza
        popup='Partenza'
    ).add_to(marker_cluster)

    # Marker dropoff
    folium.Marker(
        location=(row['dropoff_latitude'], row['dropoff_longitude']),
        icon=folium.Icon(color='red', icon='flag', prefix='fa'),  # Icona per identificare il punto di arrivo
        popup='Arrivo'
    ).add_to(marker_cluster)

# Visualizza la mappa
nyc_map
# todo: problem whit association of the points

### Data visualizazion ten most pick up e drop off points (code 17)

In [ ]:
# Filter only the data for the morning
morning_data = df[df['pickup_period'] == 'morning']

# Find the 10 most frequent pickup points
top_pickups = morning_data.groupby(['pickup_latitude', 'pickup_longitude']).size().reset_index(name='counts')
top_pickups = top_pickups.nlargest(10, 'counts').sort_values(by='counts', ascending=False)  # Ordinati per frequenza

# Find the 10 most frequent dropoff points
top_dropoffs = morning_data.groupby(['dropoff_latitude', 'dropoff_longitude']).size().reset_index(name='counts')
top_dropoffs = top_dropoffs.nlargest(10, 'counts').sort_values(by='counts', ascending=False)  # Ordinati per frequenza

# Create the map
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Cluster for pickup and dropoff points (is not important in ten most pick up and drop off points)
pickup_cluster = MarkerCluster(name='Top 10 Pickup Points - Morning').add_to(nyc_map)
dropoff_cluster = MarkerCluster(name='Top 10 Dropoff Points - Morning').add_to(nyc_map)

# Add the pickup points with explicit index
f=0
for index, row in top_pickups.iterrows():
    f += 1  # Incrementa f
    folium.Marker(
        location=(row['pickup_latitude'], row['pickup_longitude']),
        icon=folium.Icon(color='blue', icon='play', prefix='fa'),
        popup=f'Questo è il numero {f} di punti di partenza (frequenza: {row["counts"]})'  # Indice corretto
    ).add_to(pickup_cluster)

# Add the dropoff points with explicit index
k = 0  
for index, row in top_dropoffs.iterrows():
    k += 1  # increments k
    folium.Marker(
        location=(row['dropoff_latitude'], row['dropoff_longitude']),
        icon=folium.Icon(color='red', icon='flag', prefix='fa'),
        popup=f'Questo è il numero {k} di punti di arrivo (frequenza: {row["counts"]})'  # Indice corretto
    ).add_to(dropoff_cluster)

# add a layer control
folium.LayerControl().add_to(nyc_map)

# show the map
nyc_map
#!!!!!! todo:  problem with the zoom of map

### Clusterinf big data visualization (code 17)

In [ ]:
# Filter only the data for the morning
morning_data = df[df['pickup_period'] == 'morning']

# Find the 100 most frequent pickup points
top_pickups = morning_data.groupby(['pickup_latitude', 'pickup_longitude']).size().reset_index(name='counts')
top_pickups = top_pickups.nlargest(100, 'counts').sort_values(by='counts', ascending=False)  # Ordinati per frequenza

# Find the 100 most frequent dropoff points
top_dropoffs = morning_data.groupby(['dropoff_latitude', 'dropoff_longitude']).size().reset_index(name='counts')
top_dropoffs = top_dropoffs.nlargest(100, 'counts').sort_values(by='counts', ascending=False)  # Ordinati per frequenza

# Create the map
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Cluster for pickup and dropoff points
pickup_cluster = MarkerCluster(name='Top 10 Pickup Points - Morning').add_to(nyc_map)
dropoff_cluster = MarkerCluster(name='Top 10 Dropoff Points - Morning').add_to(nyc_map)

# Add the pickup points with explicit index
for index, row in top_pickups.iterrows():
    folium.Marker(
        location=(row['pickup_latitude'], row['pickup_longitude']),
        icon=folium.Icon(color='blue', icon='play', prefix='fa'),
        popup=f'Punto di partenza #{index + 1} (frequenza: {row["counts"]})'  # Indice corretto
    ).add_to(pickup_cluster)

# Add the dropoff points with explicit index
k = 0 
for index, row in top_dropoffs.iterrows():
    k += 1  
    folium.Marker(
        location=(row['dropoff_latitude'], row['dropoff_longitude']),
        icon=folium.Icon(color='red', icon='flag', prefix='fa'),
        popup=f'Punto di arrivo #{k} (frequenza: {row["counts"]})'  # Indice corretto
    ).add_to(dropoff_cluster)

# 
for _, pickup in top_pickups.iterrows():
    for _, dropoff in top_dropoffs.iterrows():
        # Check if the route is in the morning data
        route_data = morning_data[
            (morning_data['pickup_latitude'] == pickup['pickup_latitude']) &
            (morning_data['pickup_longitude'] == pickup['pickup_longitude']) &
            (morning_data['dropoff_latitude'] == dropoff['dropoff_latitude']) &
            (morning_data['dropoff_longitude'] == dropoff['dropoff_longitude'])
        ]
        
        if not route_data.empty:
            folium.PolyLine(
                locations=[(pickup['pickup_latitude'], pickup['pickup_longitude']),
                           (dropoff['dropoff_latitude'], dropoff['dropoff_longitude'])],
                color='green',
                weight=2,
                opacity=0.5
            ).add_to(nyc_map)

# Add a layer control
folium.LayerControl().add_to(nyc_map)

# Show the map
nyc_map
